## Support Vector Machines
* Poderosa metodologia para resolver problemas de aprendizagem de máquina.
* Proposto em 1995 pelo russo Vladimir Vapnik.
* Muito utilizado atualmente em diversos tipos de aplicações.
* O problema dos problemas não linearmente separáveis 
* O problema dos problemas não linearmente separáveis


![](https://i.ibb.co/tsDTKzn/SVM-Picture1.png)

* Problema não linearmente separável

### O processo de classificação:
  + Entendendo o parâmetro C, Soft Margin: 
    - A aplicação de um método puramente linear para classificar um conjunto de dados pode sofrer com dois problemas bastante comuns:
      - Outliers
      - Exemplos rotulados erroneamente
  + Mesmo assim o SVM ainda assim pode ser aplicado através do uso do parâmetro C (soft margin - variáveis de folga)
#### O truque do Kernel:
    - Realiza a projeção das características em um espaço multidimencional, maior que o vetor de entrada onde as classes podem ser separadas por um hiperplano.

![](https://i.ibb.co/n19dmvq/SVM-Picture2.png)

#### Vantagens:
  + Considerado o estado da arte dos classificadores.
  + Trabalha com problemas linearmente e não-linearmente separáveis.
  + Desempenho superior a maioria dos classificadores tradicionais.
  
#### Desvantagens:
  + A calibragem dos parâmetros envolvidos e escolha do tipo de kernel não é uma tarefa trivial.
  + O tempo de treinamento pode ser extremamente lento e computacionalmente complexo dependendo da quantidade de características.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt 

from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn import svm
from sklearn.preprocessing import LabelEncoder

# Importe as bibliotecas de Pipelines e Pré-processadores
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Importa o pacote OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

import sys

## 1) Importando os dados já tratados sem OneHotEncoding:

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls "/content/drive/My Drive/datasets"
# data = pd.read_csv('/content/drive/My Drive/datasets/dados_clust_v2.csv',sep=',') # faz a leitura dos dados

data = pd.read_csv("/Users/andrerodrigues/Google Drive/FIAP-MBA 8IA/006 - Modelos de IA e ML/Trabalho_Final_2/Exercicio8_v2/dados_clust_v2.csv", sep =",")

data = data.iloc[:,1:]
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,dbscan,kmeans,som
0,-122.23,452600.0,129,41,880,3,126,322,83.252,37.88,1,3,1
1,-122.22,358500.0,1106,21,7099,3,1138,2401,83.014,37.86,0,2,2
2,-122.24,352100.0,190,52,1467,3,177,496,72.574,37.85,1,3,1
3,-122.25,341300.0,235,52,1274,3,219,558,56.431,37.85,1,1,1
4,-122.25,342200.0,280,52,1627,3,259,565,38.462,37.85,1,1,1


## 2) Importando os dados já tratados com OneHotEncoding:

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls "/content/drive/My Drive/datasets"
# data_ohe = pd.read_csv('/content/drive/My Drive/datasets/data_ohe_nostd_v2.csv',sep=',') # faz a leitura dos dados

data_ohe = pd.read_csv("/Users/andrerodrigues/Google Drive/FIAP-MBA 8IA/006 - Modelos de IA e ML/Trabalho_Final_2/Exercicio8_v2/data_ohe_nostd_v2.csv", sep =",")

data_ohe = data_ohe.iloc[:,1:]
data_ohe["dbscan"] = data["dbscan"]
data_ohe["kmeans"] = data["kmeans"]
data_ohe["som"] = data["som"]
data_ohe.head()

ModuleNotFoundError: No module named 'google'

#### a) Apenas como curiosidade, vejamos como as classes estão distribuidas em 'dbscan', 'kmeans' e 'som'

In [ ]:
plt.figure(figsize=(10,7))
plot1 = sb.countplot(x='dbscan',data=data, palette='hls')
plt.title("Verificando classes de dbscan")
plt.show(plot1)
plt.figure(figsize=(10,7))
plot2 = sb.countplot(x='kmeans',data=data, palette='hls')
plt.title("Verificando classes de kmeans")
plt.show(plot2)
plt.figure(figsize=(10,7))
plot3 = sb.countplot(x='som',data=data, palette='hls')
plt.title("Verificando classes de som")
plt.show(plot3)

## 3) Seleciona o dataset para ser tratado pelo Modelo de Machine Learning e Separa as Classes

In [5]:
import traceback

def dataset_for_machine(dataset, target):
    
    # Tente receber o nome do arquivo
    try:
        d = dataset
        t = target
        
        
    
        if d == 'data':
            #No caso do DBSCAN, as amostras cujo o cluster for igual a 0, são consideradas outliers e não podemos considera-las como classe do problema.
            if t == 'dbscan':
                data.drop(data[data.dbscan == 0].index, inplace=True)
            # Troca o nome do dataset para df. A partir desse ponto, o dataset para ser tratado pelo modelo de ML será conhecido por df.
            df = data
            # Separa a classe
            classes = df[t]
            df.drop(columns=['dbscan','kmeans', 'som'], axis=1, inplace=True)


        elif d == 'data_ohe':
            #No caso do DBSCAN, as amostras cujo o cluster for igual a 0, são consideradas outliers e não podemos considera-las como classe do problema.
            if t == 'dbscan':
                data.drop(data[data.dbscan == 0].index, inplace=True)
            # Troca o nome do dataset para df. A partir desse ponto, o dataset para ser tratado pelo modelo de ML será conhecido por df.
            df = data_ohe
            # Separa a classe
            classes = df[t]
            df.drop(columns=['dbscan','kmeans', 'som'], axis=1, inplace=True)

        else:
            print("Escolha entre 'data' ou 'data_ohe'")
            exit

    # Se ocorrer um erro
    except:
        # Mostre na tela
        trace = traceback.format_exc()
        print ('Aconteceu um erro:\n', trace)
        # Encerre o programa
        raise SystemExit
        
    return df, classes
    
# df, classes = dataset_for_machine(input("Entre com o dataset que será tratado pela maquina('data' ou 'data_ohe') e o target('kmeans' ou 'som') separados por ',':"))
dataset = input("Entre com o dataset que será tratado pela maquina('data' ou 'data_ohe'):")
target = input("Entre com o target que será tratado pela maquina ('dbscan','kmeans' ou 'som'):")
df, classes = dataset_for_machine(dataset, target)

Entre com o dataset que será tratado pela maquina('data' ou 'data_ohe'): data_ohe
Entre com o target que será tratado pela maquina ('dbscan','kmeans' ou 'som'): som


* A partir desse ponto, o dataset para ser tratado pelo modelo de ML será conhecido por df. A seleção se ele vai ser passado com ou sem One Hot Encoding foi feita acima.

In [6]:
df.head()

,0,1,2,3,4,6,7,8,9,10,11,12,13,14
0,-122.23,452600.0,129.0,41.0,880.0,126.0,322.0,83.252,37.88,0.0,0.0,0.0,1.0,0.0
1,-122.22,358500.0,1106.0,21.0,7099.0,1138.0,2401.0,83.014,37.86,0.0,0.0,0.0,1.0,0.0
2,-122.24,352100.0,190.0,52.0,1467.0,177.0,496.0,72.574,37.85,0.0,0.0,0.0,1.0,0.0
3,-122.25,341300.0,235.0,52.0,1274.0,219.0,558.0,56.431,37.85,0.0,0.0,0.0,1.0,0.0
4,-122.25,342200.0,280.0,52.0,1627.0,259.0,565.0,38.462,37.85,0.0,0.0,0.0,1.0,0.0


In [7]:
classes

0         1
1        13
2         1
3        21
4        21
         ..
20635     2
20636     2
20637     2
20638     2
20639     5
Name: kmeans, Length: 20640, dtype: int64

## 4) Faz a validação via crossvalidation (k-fold)

In [8]:
def Acuracia(clf,X,y):
    resultados = cross_val_predict(clf, X, y, cv=5)
    return metrics.accuracy_score(y,resultados)

## 5) Pre-processamento de dados

In [9]:
# def remove_features(lista_features):
#     for i in lista_features:
#         data.drop(i, axis=1, inplace=True)
#     return 0

## 9) Support Vector Machines 
#### a) Preparando os pipes com as configurações do parâmetro C, Kernel e Normalização dos dados.

In [10]:
pip_1 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC())
])

pip_2 = Pipeline([
    ('min_max_scaler', MinMaxScaler()),
    ('clf', svm.SVC())
])

pip_3 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC(kernel='rbf'))
])

pip_4 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC(kernel='poly'))
])

pip_5 = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', svm.SVC(kernel='linear'))
])

#### b) Teste com apenas LabelEncoder na coluna 'classes' usando o pipeline 'pip_1'

In [11]:
Acuracia(pip_1,df,classes)

/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9186046511627907

#### c) Teste com apenas LabelEncoder na coluna 'classes' usando o pipeline 'pip_2'

In [12]:
Acuracia(pip_2,df,classes)

/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/andrerodr

0.7504360465116279

#### d) Testando Kernels 
##### Kernel rbf

In [13]:
Acuracia(pip_3,df,classes)

/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9186046511627907

##### Kernel Polynomial

In [14]:
Acuracia(pip_4,df,classes)

/Users/andrerodrigues/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9044573643410853

##### Kernel Linear

In [15]:
Acuracia(pip_5,df,classes)

0.9455910852713179

## 9) GridSearch

In [16]:
from sklearn.model_selection import GridSearchCV

lista_C = [0.001, 0.01, 0.1, 1, 10,100]
lista_gamma = [0.001, 0.01, 0.1, 1, 10, 100]

In [17]:
parametros_grid = dict(clf__C=lista_C, clf__gamma=lista_gamma)

#### a) Faz o tuning dos parametros testando cada combinação utilziando CrossValidation com 10 folds e analisando a acurácia

In [19]:
grid = GridSearchCV(pip_5, parametros_grid, cv=10, scoring='accuracy')

In [20]:
grid.fit(df,classes)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        SVC(C=1.0, cache_size=200,
                                            class_weight=None, coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='auto_deprecated',
                                            kernel='linear', max_iter=-1,
                                            probability=False,
                                            random_state=None, shrinking=True,
                                            tol=0.001, verbose=False))],
            

In [21]:
grid.cv_results_

{'mean_fit_time': array([3.48232725, 3.4139832 , 3.43661852, 3.41013134, 3.4734334 ,
        3.43143516, 1.11112182, 1.10829029, 1.10815685, 1.11026218,
        1.10803168, 1.10285707, 0.5703167 , 0.56714962, 0.5791609 ,
        0.56898217, 0.57245896, 0.58555398, 0.50374458, 0.50886505,
        0.50485389, 0.5054831 , 0.50379093, 0.50665677, 0.59823294,
        0.59786091, 0.598387  , 0.60060146, 0.63420999, 0.70456119,
        0.75283442, 0.68955264, 0.73888581, 0.67441404, 0.77003953,
        0.66647525]),
 'std_fit_time': array([0.16329294, 0.03644745, 0.06817607, 0.05621684, 0.18040119,
        0.05485186, 0.01593242, 0.01458568, 0.01769101, 0.01102343,
        0.01210841, 0.01355088, 0.00750296, 0.0067833 , 0.01778375,
        0.00770169, 0.01020969, 0.03965704, 0.01505802, 0.01796968,
        0.01683742, 0.01254418, 0.01563671, 0.01918492, 0.01433831,
        0.0143621 , 0.0149577 , 0.01793844, 0.05148708, 0.06372564,
        0.07386091, 0.0421662 , 0.06935114, 0.03748293, 0.068

In [22]:
grid.best_params_

{'clf__C': 100, 'clf__gamma': 0.001}

In [23]:
grid.best_score_

0.9739341085271318

#### b) Métricas de Avaliação de Modelos

In [24]:
pip_6 = Pipeline([
('scaler',StandardScaler()),
('clf', svm.SVC(kernel='linear',C=100,gamma=0.001))
])

In [25]:
resultados = cross_val_predict(pip_6, df, classes, cv=10)

In [27]:
c_names = [str(i) for i in np.unique(classes)]
print (metrics.classification_report(classes,resultados,target_names=c_names))

              precision    recall  f1-score   support

           1       0.95      0.98      0.96       343
           2       0.98      0.99      0.99      1340
           3       0.98      0.98      0.98      1190
           4       0.95      0.96      0.95       397
           5       0.96      0.97      0.97      1061
           6       0.98      0.98      0.98      1676
           7       0.94      0.99      0.96       447
           8       0.96      0.97      0.96       887
           9       0.98      0.98      0.98      1303
          10       0.96      0.95      0.96       627
          11       0.98      0.98      0.98       883
          12       0.97      0.95      0.96       369
          13       0.97      0.94      0.95       477
          14       0.95      0.97      0.96       998
          15       0.98      0.97      0.98       827
          16       0.89      0.97      0.93        32
          17       0.98      0.97      0.98      1499
          18       0.99    